In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms


In [ ]:

# Define the model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 4 * 4, 256)  # Adjusted input size after pooling
        self.fc2 = nn.Linear(256, 10)  # Output will be 10 classes for CIFAR-10

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)  # Adding dropout for regularization

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x))
        x = self.pool(x)
        x = x.view(-1, 64 * 4 * 4)  # Adjusted size after pooling
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x



In [ ]:
# Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to range [-1, 1]
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)




100%|██████████| 170498071/170498071 [00:02<00:00, 60543647.83it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
from torch.optim.lr_scheduler import StepLR

# Define the model, loss function, and optimizer
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


In [ ]:
import matplotlib.pyplot as plt

loss_values = []

for epoch in range(5):  # Adjust number of epochs as needed
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 200 == 199:  # Print every 200 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            loss_values.append(running_loss / 200)
            running_loss = 0.0

print('Finished Training')

# Plot the loss graph
plt.plot(loss_values)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.show()


In [ ]:
correct = 0
total = 0

# Disable gradient calculation for evaluation
model.eval()

with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy on the test set: %d %%' % (100 * correct / total))


Accuracy on the test set: 65 %


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# Define the model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 4 * 4, 256)  # Adjusted input size after pooling
        self.fc2 = nn.Linear(256, 10)  # Output will be 10 classes for CIFAR-10
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.5)  # Adding dropout for regularization

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(-1, 64 * 4 * 4)  # Adjusted size after pooling
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

# Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to range [-1, 1]
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)

# Define the model, loss function, and optimizer
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 5
loss_values = []
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 200 == 199:  # Print every 200 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 200))
            loss_values.append(running_loss / 200)
            running_loss = 0.0

print('Finished Training')

# Plot the loss graph
plt.plot(loss_values)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.show()

# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print('Accuracy on the test set: %.2f %%' % accuracy)


In [ ]:
import numpy as np
import time
# Record the start time
start_time = time.time()

# Call your function here

def convolve2d(input_matrix, kernel):
    # Get the dimensions of the input matrix and kernel
    input_height, input_width = input_matrix.shape
    kernel_height, kernel_width = kernel.shape

    # Compute the output size
    output_height = input_height - kernel_height + 1
    output_width = input_width - kernel_width + 1

    # Initialize the output feature map
    output = np.zeros((output_height, output_width))

    # Flip the kernel (required for convolution)
    kernel = np.flipud(np.fliplr(kernel))

    # Perform 2D convolution
    for i in range(output_height):
        for j in range(output_width):
            output[i, j] = np.sum(input_matrix[i:i+kernel_height, j:j+kernel_width] * kernel)

    return output

# Example usage:
input_matrix = np.array([[0,0, 0, 0,0],
                         [0,1, 2, 3,0],
                         [0,4, 5, 6,0],
                         [0,7, 8, 9,0],
                         [0,0, 0, 0,0]])

kernel = np.array([[1, 0, -1],
                   [2, 0, -2],
                   [1, 0, -1]])

output_feature_map = convolve2d(input_matrix, kernel)
print("Output Feature Map:")
print(output_feature_map)

# Record the end time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time, "seconds")


Output Feature Map:
[[  9.   6.  -9.]
 [ 20.   8. -20.]
 [ 21.   6. -21.]]
Elapsed time: 0.002889871597290039 seconds


In [ ]:
start_time = time.time()
a = np.convolve([1, 2, 3,4, 5, 6,7, 8, 9],[1, 0, -1,2, 0, -2,1, 0, -1])
print(a)
end_time = time.time()
elapsed_time = end_time - start_time
print("Elapsed time:", elapsed_time, "seconds")

[  1   2   2   4   6   6   7   8   8  -2  -3   6 -14 -16   2  -8  -9]
Elapsed time: 0.0007612705230712891 seconds


In [ ]:
# [[[0,0,0,0,0,0],
#                               [0,0,0,0,0,0],
#                               [0,0,0,0,0,0]],
#                              [[0,0,0,0,0,0],
#                               [0,0,0,0,0,0],
#                               [0,0,0,0,0,0]],
#                              [[0,0,0,0,0,0],
#                               [0,0,0,0,0,0],
#                               [0,0,0,0,0,0]],
#                              [[0,0,0,0,0,0],
#                               [0,0,0,0,0,0],
#                               [0,0,0,0,0,0]]

#                             ,[[1,1,1,1,1,1],
#                               [1,1,1,1,1,1],
#                               [1,1,1,1,1,1]]
#                               ,[[1,1,1,1,1,1],
#                               [1,1,1,1,1,1],
#                               [1,1,1,1,1,1]]
#                               ,[[1,1,1,1,1,1],
#                               [1,1,1,1,1,1],
#                               [1,1,1,1,1,1]]
#                               ,[[1,1,1,1,1,1],
#                               [1,1,1,1,1,1],
#                               [1,1,1,1,1,1]]

#                             ,[[2,2,2,2,2,2],
#                               [2,2,2,2,2,2],
#                               [2,2,2,2,2,2]]
#                               ,[[2,2,2,2,2,2],
#                               [2,2,2,2,2,2],
#                               [2,2,2,2,2,2]]
#                               ,[[2,2,2,2,2,2],
#                               [2,2,2,2,2,2],
#                               [2,2,2,2,2,2]]
#                               ,[[2,2,2,2,2,2],
#                               [2,2,2,2,2,2],
#                               [2,2,2,2,2,2]]
#                               ]

In [29]:
input_array = [[[0]],[[0]],[[0]],[[0]],[[1]],[[1]],[[1]],[[1]],[[2]],[[2]],[[2]],[[2]]]
input_array = np.array(input_array)
print(input_array.shape)
input_array[0].shape

(12, 1, 1)


(1, 1)

In [23]:
# Define the input array
input_array = [[[0]],[[0]],[[0]],[[0]],[[1]],[[1]],[[1]],[[1]],[[2]],[[2]],[[2]],[[2]]]

# Define the group size
group_size = 3

# Initialize the resulting array
result_array = []

# Loop through the input array in chunks of group_size
for i in range(0, len(input_array), group_size):
    # Extract the chunk
    chunk = input_array[i:i+group_size]
    # Rotate the chunk by one position
    rotated_chunk = chunk[1:] + chunk[:1]
    # Append the rotated chunk to the resulting array
    result_array.extend(rotated_chunk)

print(result_array)


[[[0]], [[0]], [[0]], [[1]], [[1]], [[0]], [[1]], [[2]], [[1]], [[2]], [[2]], [[2]]]
